In [1]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    get_load_config_from_yaml,
)
from importlib import reload
import core
import keras
import core.keras_models.RegressionTransformer as RegressionTransformer

PLOTS_DIR = f"plots/regresion_transformer/"
MODEL_DIR = f"models/regresion_transformer/"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


data_config = DataProcessor.load_from_npz(
    load_config.data_path["nominal"], max_events=400_000, event_numbers="even"
)

X_train, y_train = DataProcessor.get_data()
del DataProcessor  # Free memory

2026-01-28 14:04:04.805823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769605444.829040  335679 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769605444.836848  335679 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769605444.855852  335679 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769605444.855872  335679 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769605444.855875  335679 computation_placer.cc:177] computation placer alr

In [2]:
reload(RegressionTransformer)
Transformer = RegressionTransformer.FeatureConcatFullReconstructor(data_config)

In [3]:
Transformer.build_model(
    hidden_dim=128,
    num_layers=4,
    dropout_rate=0.1,
)

I0000 00:00:1769605542.006804  335679 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14287 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


In [4]:
Transformer.add_reco_mass_deviation_loss()

Added physics-informed loss to the model.


In [5]:
Transformer.adapt_normalization_layers(X_train)

Transformer.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(lambda_excl=0.0),
        "normalized_regression": core.utils.RegressionLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-5),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy(name="assignment_accuracy")],
        "normalized_regression": [core.utils.RegressionDeviation()],
    },
)


I0000 00:00:1769605543.802245  337990 service.cc:152] XLA service 0x7f7b3c003520 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1769605543.802270  337990 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2026-01-28 14:05:43.813393: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1769605543.828949  337990 cuda_dnn.cc:529] Loaded cuDNN version 91500
I0000 00:00:1769605543.968375  337990 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Adapted normalization layer:  lep_input_normalization
Adapted normalization layer:  jet_input_normalization


2026-01-28 14:05:45.455754: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator pre_met_input_normalization_model_1/met_input_transform_1/assert_equal_1/Assert/Assert
2026-01-28 14:05:45.812868: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator pre_met_input_normalization_model_1/met_input_transform_1/assert_equal_1/Assert/Assert


Adapted normalization layer:  met_input_normalization
Set regression layer stats.
Compiling model with physics informed loss. Ensure that the loss dictionary includes 'reco_mass_deviation'.


In [6]:
Transformer.trainable_model.loss

{'assignment': <core.utils.losses.AssignmentLoss at 0x7f7c881935e0>,
 'normalized_regression': <core.utils.losses.RegressionLoss at 0x7f7c88193c10>,
 'reco_mass_deviation': <function core.reconstruction.reconstruction_base.KerasFFRecoBase.compile_model.<locals>.<lambda>(y_true, y_pred)>}

In [7]:
Transformer.train_model(
    epochs=1,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            verbose=1,
            mode="min",
            min_lr=1e-6,
        ),
    ],
)

2026-01-28 14:06:04.846530: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator functional_1/met_input_transform_1/assert_equal_1/Assert/Assert
2026-01-28 14:06:28.896782: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_reduce_slice_fusion_1', 44 bytes spill stores, 44 bytes spill loads



311/313 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - assignment_assignment_accuracy: 0.0691 - assignment_loss: 0.2294 - loss: 2.2674 - normalized_regression_deviation: 0.5305 - normalized_regression_loss: 0.9786 - reco_mass_deviation_loss: 1.0595

2026-01-28 14:06:42.644745: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator functional_1/met_input_transform_1/assert_equal_1/Assert/Assert
2026-01-28 14:07:03.577333: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_multiply_reduce_fusion_3', 36 bytes spill stores, 32 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_2', 128 bytes spill stores, 128 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_1', 128 bytes spill stores, 128 bytes spill loads



313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - assignment_assignment_accuracy: 0.0693 - assignment_loss: 0.2293 - loss: 2.2667 - normalized_regression_deviation: 0.5305 - normalized_regression_loss: 0.9783 - reco_mass_deviation_loss: 1.0591

2026-01-28 14:07:05.334814: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator functional_1/met_input_transform_1/assert_equal_1/Assert/Assert
2026-01-28 14:07:07.863262: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator functional_1/met_input_transform_1/assert_equal_1/Assert/Assert


313/313 ━━━━━━━━━━━━━━━━━━━━ 83s 129ms/step - assignment_assignment_accuracy: 0.0693 - assignment_loss: 0.2293 - loss: 2.2664 - normalized_regression_deviation: 0.5305 - normalized_regression_loss: 0.9782 - reco_mass_deviation_loss: 1.0589 - val_assignment_assignment_accuracy: 0.1921 - val_assignment_loss: 0.1893 - val_loss: 1.9055 - val_normalized_regression_deviation: 0.4843 - val_normalized_regression_loss: 0.8078 - val_reco_mass_deviation_loss: 0.9048 - learning_rate: 1.0000e-04


In [8]:
Transformer.save_model(f"{MODEL_DIR}/model.keras")

Model saved to models/regresion_transformer//model.keras


In [10]:
MODEL_DIR = "../models/regression_transformer_pil/"

Transformer.load_model(f"{MODEL_DIR}/model.keras")

TypeError: <class 'core.base_classes.machine_learning_base.KerasModelWrapper'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'core.base_classes.machine_learning_base', 'class_name': 'KerasModelWrapper', 'config': {}, 'registered_name': 'Custom>KerasModelWrapper', 'build_config': {'input_shape': None}, 'compile_config': {}}.

Exception encountered: Could not locate class 'OutputUpScaleLayer'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'core.components.output_layers', 'class_name': 'OutputUpScaleLayer', 'config': {'name': 'regression', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None, 'shared_object_id': 140268667830576}}, 'registered_name': '